# M4L2: Record Linkage

We saw that under ideal conditions in a database, we can use the primary key to uniquely identify a record.  But in the real world, that's not always the case.

Record Linkage or Entity Resolution is the process of identifying records that refer to the same real world entity across multiple data sources.

*Deduplication* is a special case of record linkage where you have a single data source and you want to remove duplicate records. (Finding the same record refered to in multiple ways within one data source.)

## Example

You have a list of restaurants whose owners have given money to a political campaign. You also have a list of restaurants that have been fined by the city. As part of an analysis, you want to see if there is a difference in the average fine amount for businesses that have given money to the campaign versus those that have not.

| Business Name | Address | Amount Donated |
|---------------|---------|----------------|
| Little Coco's | 123 Main St | \$1000 |
| Taqueria Habanero | 901 17th St | \$250 |
| Red Derby | 917 Queen Anne Ave | \$2000 |
| Susana's | 8312 Park Blvd | \$500 |


| ID | Business Name | Address | Fine Amount |
|--| --------------|---------|-------------|
|1230| Little Coco's Restaurant LLC | 123 Main Street | \$100 |
|2901| Taqueria Habañero | 901 17th Street \#1 | \$100 |
|3014| IHOP | 917 Queen Ann Avenue | \$5000 |
|3207| Susanna's | 8312 Park Blvd | \$100 |


### Problems

There's no shared key between the two tables.  Even with IDs, the campaign finance & restaurant fine systems would not use the same IDs.

Furthermore, there are inconsistencies in how the restaurants and addresses are written. It would appear likely that "Little Coco's" and "Little Coco's Restaurant LLC" are the same restaurant, but trying to match on columns directly will not work.

### Record Linkage Steps

1. Identify the fields that you want to link on (link keys).   

2. Clean & standardize the data in those fields.  (e.g. remove punctuation, convert to lowercase, etc.)

3. Score similarity between each link key pair.

4. Threshold the scores to determine which pairs are similar enough to be linked.

Step 3 requires us to compare every record to every other record.

Basic form:

```python
matches = []

for item1 in dataset1:
    for item2 in dataset2:
        # thresholds were chosen arbitrarily
        if item_similarity(item1.name, item2.name) > 0.8 and items_similarity(item1.address, item2.address) > 0.9:
            matches.append((item1, item2))
```

Note: `item_similarity` is a function you'd need to define, however you want to compare the strings.

### Aside: `itertools.product`

When you have a nested loop, you can use `itertools.product` to make it a bit more concise.  

`itertools.product` takes two iterables and returns an iterable of tuples of the items from each iterable.  (In relational terms, the cross product of the two iterables.)

In [ ]:
import itertools

list1 = ["a", "b", "c"]
list2 = [1, 2, 3]
for item1, item2 in itertools.product(list1, list2):
    print(item1, item2)

```python
import itertools

matches = []
for item1, item2 in itertools.product(dataset1, dataset2):
    if item_similarity(item1.name, item2.name) > 0.8 and item_similarity(item1.address, item2.address) > 0.9:
        matches.append((item1, item2))
```

### Efficiency

Without clever tricks, this algorithm is `O(n*m)` where `n` is the number of items in `dataset1` and `m` is the number of items in `dataset2`.

This is effectively `O(n^2)` because `n` and `m` are usually similar enough.

That means when you have 1000 items in each dataset, you're doing 1,000,000 calls to your `item_similarity()` function.

But `item_similarity()` is fast right??

## String Similarity

There are a lot of ways to compare strings.

### Hamming Distance

Computes the number of characters that differ between two strings of equal length.

```python
def hamming_distance(s1, s2):
    # ensure length of s1 >= s2 using an in-place swap
    if len(s2) > len(s1):
        s1, s2 = s2, s1

    # distance is difference in length + differing chars
    distance = len(s1) - len(s2)
    for i, c in enumerate(s2):
        if c != s1[i]:
            distance += 1

    return distance
```

This is a very simple implementation, but as we'll see it isn't incredibly useful:

In [ ]:
from jellyfish import hamming_distance

print(hamming_distance("Taqueria Habañero", "Taqueria Habanero"))
1
print(hamming_distance("Susana's", "Susanna's"))
4

Both of these are one character off, but the Hamming distance is 1 and 4, respectively.

Hamming distance only handles replacements, not insertions or deletions. Since it is not normalized to the length of the string, it works best on strings of the same length.

### Levenshtein Distance

Levenshtein distance is a more general version of Hamming distance. It allows for insertions, deletions, and replacements.

The bad news is that the algorithm is O(n*m) where n and m are the lengths of the strings.

In [ ]:
from jellyfish import levenshtein_distance

print(levenshtein_distance("Taqueria Habañero", "Taqueria Habanero"))
1
print(levenshtein_distance("Susana's", "Susanna's"))
1

Levenshtein distance is also known as edit distance.

#### Damerau-Levenshtein Distance

Adjustment to Levenshtein distance that allows for transpositions to be counted as a single edit.

In [ ]:
from jellyfish import damerau_levenshtein_distance

print(levenshtein_distance("Fish", "Fsih"))
2
print(damerau_levenshtein_distance("Fish", "Fsih"))
1

But we still have the issue that strings of different lengths are not comparable.

In [ ]:
print(damerau_levenshtein_distance("Little Coco's", "Little Coco's Restaurant LLC"))
15

It'd be nice if these were somehow normalized based on length.

### Jaro Similarity

The Jaro distance is a string similarity measure, which measures the edit distance between two sequences.

It **normalizes** the distance by the length of the strings so that the result is between 0 and 1.

In [ ]:
from jellyfish import jaro_similarity

print(jaro_similarity("Taqueria Habañero", "Taqueria Habanero"))
# 0.9608

print(jaro_similarity("Susana's", "Susanna's"))
# 0.9630

print(jaro_similarity("Little Coco's", "Little Coco's Restaurant LLC"))
# 0.8214

print(jaro_similarity("Little Coco's", "Susanna's"))
# 0.4587

#### Jaro-Winkler 
And the Jaro-Winkler similarity is a modification of the Jaro distance that gives more favorable ratings to strings that match from the beginning.

In [ ]:
from jellyfish import jaro_winkler_similarity

print(jaro_winkler_similarity("Taqueria Habañero", "Taqueria Habanero"))
# 0.9707

print(jaro_winkler_similarity("Susana's", "Susanna's"))
# 0.9778

print(jaro_winkler_similarity("Little Coco's", "Little Coco's Restaurant LLC"))
# 0.8929

print(jaro_winkler_similarity("Little Coco's", "Susanna's"))
# 0.4587

All of our matches got a boost from Jaro-Winkler similarity.

If the beginning of a string is more important than the end, this makes Jaro-Winkler a good choice.

### String Similarity: Conclusion

Hamming, Levenshtein, and Damerau-Levenshtein can all be useful in different situations.

Jaro or Jaro-Winkler make a good general purpose choice.

## Phonetic Encoding

An alternative approach is to shorten the string by encoding it in different ways.

### Match Rating Approach

A generic version of this is known as the Match Rating Approach.  This converts each string into a string of characters that are similar to the original string based upon phonetic similarity.

Example:

In [ ]:
from jellyfish import match_rating_codex, match_rating_comparison

print(match_rating_codex("Taqueria Habañero"))
# TQRBÑR
print(match_rating_codex("Susana's"))
# SNSS
print(match_rating_comparison("Taqueria Habañero", "Taqueria Habanero"))
True

### Soundex

Soundex is a phonetic algorithm for indexing names by sound, as pronounced in English. The goal is for homophones to be encoded to the same representation so that they can be matched despite minor differences in spelling.

`soundex('Ann') == soundex('Anne') == 'A500'`

`soundex('Rupert') == soundex('Robert') == 'R163'`


### Metaphone

Metaphone is more complex than Soundex, but it's also more accurate.

`metaphone('Klumpz') == metaphone('Clumps') == 'KLMPS'.`

Metaphone is particularly good at handling soundalikes like C/K and PH/F.

### And So On...

* NYSIIS
* Double Metaphone
* Triple Metaphone
* Caverphone - New Zealand Pronunciation

## How Similar is Similar Enough?

How do you decide what is similar enough?

In [ ]:
import itertools
import matplotlib.pyplot as plt
from jellyfish import jaro_winkler_similarity

all_words = open("shakespeare.txt").read().split()
words = all_words[:2000]

def graph_jw(list1, list2):
    all_similarities = []
    for word1, word2 in itertools.product(list1, list2):
        jws = jaro_winkler_similarity(word1, word2)
        if jws > 0:
            all_similarities.append(jws)
    # graph of all non-zero similarities
    plt.hist(all_similarities, bins=100)

graph_jw(words, words)

This is a histogram of the Jaro-Winkler similarity scores for all pairs of words in the first 2000 words of Shakespeare's works. 

The vast majority of words fall between 0.4 and 0.7 or so, you'll want to select a threshold that makes sense for your data.

Graphing output like this can be useful in developing some intuition about your particular data set.

(**See Jaro-Winkler PDF**)

## Record Linkage

With string similarity in our toolbox, we can now start to think about how to use it to link records.

### Record Linkage Steps Revisited

1. Identify the fields that you want to link on (link keys).   

2. Clean & standardize the data in those fields.  (e.g. remove punctuation, convert to lowercase, etc.)

3. Score similarity between each link key pair.

4. Threshold the scores to determine which pairs are similar enough to be linked.

We've now covered steps 1-3.  Let's look at step 4.  How do we decide what is similar enough?

### Rules Based Approaches

One approach is to use rules based on the data.  For example, if you're linking on names, you might say that if the Jaro-Winkler similarity is greater than 0.9, then the records are linked.

This requires making arbitrary decisions about what is similar enough.  This is often done by running with different thresholds and looking at the results until one is found that seems reasonable.

Example:

In [ ]:
from jellyfish import jaro_winkler_similarity

def link_records(record1, record2):
    return (jaro_winkler_similarity(record1['name'], record2['name']) > 0.9 and
            jaro_winkler_similarity(record1['address'], record2['address']) > 0.9 and
            record1['city'] == record2['city'] and
            record1['state'] == record2['state'])

### Probabilistic Approaches

A more principled approach is to use a probabilistic model to determine the likelihood that two records are linked.

This is a bit more complicated, but it has the advantage that it can be used to determine the optimal threshold for a specific application.

For example, if you are dealing with linking medical records, you might want to be conservative in what you link to avoid accidentally linking two patients who are not the same person.  In this case, you would want to configure your model to have a low false positive rate.

On the other hand, if you are doing an analysis across decennial census records, you might want to be more aggressive in linking records to get a more complete picture of the population.  In this case, you would want to configure your model to have a low false negative rate.

Probabilistic approaches require already labeled data to train the model. This is often done by having a human look at a sample of the records and label them as linked or not linked.  This data will form a baseline that the model's predictions can be compared against.

### Probabilistic Approaches: Example

We have two lists of people that we'd like to link: `A` and `B`.

We take a sample of the data `A_sample` and `B_sample` and label the pairs (`A_sample X B_sample`) as linked or not linked.  Call the matched pairs `M` and the unmatched pairs `U`.

We then define a scoring function for each link key.  For example, we might have a scoring function for names that returns "good match", "intermediate match", or "no match".  We'll call this `name_scoring_function`.  We could also have a similar function for address, year of birth, etc.

If we get two records `a` and `b` from `A` and `B` respectively, we'd use our scoring function:

```python
name_score = scoring_function(a["name"], b["name"])
# score is either "good match", "intermediate match", or "no match"
```

Let's say we get "intermediate match".

We need to know the probability of getting such a score among the matches and among the non-matches.  We can compute this from the labeled data.

We determine the probability of getting such a value among the matches in `M`, `P(name has "intermediate" | M)`, by looking at the fraction of matches that have such a value for the field name.

Similarly we determine the probability for getting such a value among the non-matches, `P(first name has "intermediate" | U)`.

We compute such probabilities corresponding to address as well.

Suppose the result of `address_scoring_function(a, b)` is `"good match"`.
We then compute `P(address has "good match" | M)` and `P(address has "good match" | U)`.

Now we are ready to find the ratio of the probabilities, assuming independence between first name and year of birth.

```R = ( P(name has "intermediate | M) * P(address has "good match" | M) ) / ( P(name has "intermediate | U) * P(address has "good match" | U )```

Based again on training data we determine thresholds T1 and T2, such that when the score is above T1 we assign (a, b) as a match, and if below T2 as a non-match.  In between them they are sent for a manual clerical review.

### Machine Learning Approaches

Another approach is to use machine learning to determine the optimal threshold.  This is particularly useful when there are a large number of fields and you don't have a clear idea of how they help in record linkage.  It's beyond the scope of what we can cover here, but relatively easy to implement using a library like scikit-learn.

<https://scikit-learn.org/stable/index.html>

### Blocking

Sometimes there are fields that we are confident are very important for a match.

Perhaps 99.9% of matches have the correct state. If when comparing two records we find that they have different states, we can be confident that they are not a match and avoid doing the rest of the work.

Let's say we have a million records in each dataset, no matter how fast our algorithm is, we're looking at a trillion comparisons.

If we first reduce the records by state, into 50 groups averaging 200,000 records each, "only" 20 billion comparisons need to be done.

That's a 98% reduction in the number of comparisons.

You can get clever by creating a blocking key, which is a string that is used to group records together.

For example, you could use the first letter of a person's last name and their zip code to group people in a dataset so you're only comparing records
that have a fairly high likelihood of matching.

### Privacy Issues

Record linkage can be used to de-anonymize data.

E.g., a Harvard researcher was able to identify 40% of anonymous donors in a DNA study because the data included zip codes, dates of birth, and genders, and the researcher combined these with voter rolls, etc.

When linking records, it's important to consider the privacy implications of the data you're linking, particularly when individuals are involved.

As always, practice responsible disclosure, if you uncover a privacy issue, report it to the data owner with suggestions on how to remedy it. Careless de-anonymization can have serious consequences for vulnerable individuals.